# Final Project - Main Notebook


So, for this project, I decided to fit a two-stage model of interocular suppression (described here: http://jov.arvojournals.org/article.aspx?articleid=2122317) to some psychophysics data, then visualize the results. I hoped to do the fitting using the procedure described here: http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0150942#sec019.

I developed the code in this notebook, then moved it over to some python files as it reached completion. There are roughly three parts:
 * Utils (utils.py) for reading in the data, converting it to a data frame, grouping and summarizing conditions, and fitting the model.
  * This stuff was relatively straightforward.
  * There are two tasks, Orientation Suppression and Surround Suppression. I wrote my code to be as general as possible so it could be applied to both.
 * The model, implemented in two_stage_model.py.
  * This took wayyyy too long, because I got bogged down in the deatails of the model and the fitting procedure
 * The visualization code, which resides in plotting.py
  * This got short shrift, because I spent too long fitting the model

Run the cells in this notebook to re-do the analysis.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from bokeh.plotting import output_notebook, show

# functions written for this project
import utils
import two_stage_model as model
import plotting

### Orientation Suppression Task

In [3]:
os_df = utils.load_all_data("cleanData/*filteredData_OS.mat", utils.load_individual_os_data)

In [4]:
gvars = ["Subject", "Eye", "Orientation", "Presentation"] # Presentation conditions
gvars_mask = gvars + ["MaskContrast"] # Mask contrast, m in the model
gvars_masktarget = gvars_mask + ["ProbeContrastUsed"] # Target contrast, t in the model

#### Summarize the data at the level we need

In [5]:
grouped, condensed_df = utils.summarize_conditions(os_df, gvars_masktarget)

In [6]:
df = condensed_df.reset_index()
df.head(n=10)

,Subject,Eye,Orientation,Presentation,MaskContrast,ProbeContrastUsed,n,c,pct_correct
0,ah,1,0,1,11.0,0.82,2,1,0.500000
1,ah,1,0,1,11.0,1.10,3,2,0.666667
2,ah,1,0,1,11.0,1.47,7,3,0.428571
3,ah,1,0,1,11.0,1.96,15,11,0.733333
4,ah,1,0,1,11.0,2.61,19,12,0.631579
5,ah,1,0,1,11.0,3.48,26,22,0.846154
6,ah,1,0,1,11.0,4.64,24,16,0.666667
7,ah,1,0,1,11.0,6.19,32,25,0.781250
8,ah,1,0,1,11.0,8.25,26,24,0.923077
9,ah,1,0,1,11.0,11.00,22,21,0.954545


#### Fit the model



In [7]:
params = model.two_stage_parameters()

In [8]:
os_fitted_params = df.groupby(gvars).apply(utils.model_condition, model.two_stage_response, params)

('ah', 1, 0, 1) 54
('ah', 1, 0, 1) 54
('ah', 1, 0, 2) 35
('ah', 1, 90, 1) 53
('ah', 1, 90, 2) 49
('ah', 2, 0, 1) 56
('ah', 2, 0, 2) 63
('ah', 2, 90, 1) 55
('ah', 2, 90, 2) 62
('ai', 1, 0, 1) 41
('ai', 1, 0, 2) 47
('ai', 1, 90, 1) 30
('ai', 1, 90, 2) 37
('ai', 2, 0, 1) 32
('ai', 2, 0, 2) 37
('ai', 2, 90, 1) 23
('ai', 2, 90, 2) 30
('am', 1, 0, 1) 59
('am', 1, 0, 2) 51
('am', 1, 90, 1) 53
('am', 1, 90, 2) 95
('am', 2, 0, 1) 58
('am', 2, 0, 2) 76
('am', 2, 90, 1) 53
('am', 2, 90, 2) 70
('ap', 2, 0, 1) 53
('ap', 2, 0, 2) 38
('ap', 2, 90, 1) 51
('ap', 2, 90, 2) 41
('bi', 1, 0, 1) 37
('bi', 1, 0, 2) 30
('bi', 1, 90, 1) 36
('bi', 1, 90, 2) 36
('bi', 2, 0, 1) 34
('bi', 2, 0, 2) 37
('bi', 2, 90, 1) 26
('bi', 2, 90, 2) 37
('bo', 1, 0, 1) 52
('bo', 1, 0, 2) 59
('bo', 1, 90, 1) 49
('bo', 1, 90, 2) 51
('cl', 1, 0, 1) 26
('cl', 1, 0, 2) 43
('cl', 1, 90, 1) 28
('cl', 1, 90, 2) 34
('cl', 2, 0, 1) 28
('cl', 2, 0, 2) 47
('cl', 2, 90, 1) 31
('cl', 2, 90, 2) 41
('cr', 2, 0, 1) 55
('cr', 2, 0, 2) 31
('cr', 

In [9]:
os_fitted_df = os_fitted_params.reset_index().dropna()

In [10]:
os_fitted_df.head()

,Subject,Eye,Orientation,Presentation,m,S,w_m,w_d,a,k,p,q,Z
0,ah,1,0,1,1.3,1.0,3.353600,1.000000,8.040996,0.2,8.0,6.5,0.0085
1,ah,1,0,2,1.3,1.0,1.000000,1.008823,-0.004268,0.2,8.0,6.5,0.0085
2,ah,1,90,1,1.3,1.0,0.050109,1.000000,0.586647,0.2,8.0,6.5,0.0085
3,ah,1,90,2,1.3,1.0,1.000000,1.533403,4.926649,0.2,8.0,6.5,0.0085
4,ah,2,0,1,1.3,1.0,0.563338,1.000000,1.775208,0.2,8.0,6.5,0.0085


### Visualization

In [11]:
output_notebook()
renderer, layout = plotting.plot_suppressive_weights(os_fitted_df)

Loading BokehJS ...

In [12]:
show(layout)

### Surround Suppression

In [13]:
ss_df = utils.load_all_data("cleanData/*filteredData_SS.mat", utils.load_individual_ss_data)

### Testing

In [14]:
!pytest test_utils.py

============================= test session starts ==============================
platform darwin -- Python 3.6.2, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: /Users/smerdis/code/ay250-hw/final_project, inifile:
plugins: cov-2.5.1
collected 1 item                                                                

test_utils.py .

=========================== 1 passed in 2.04 seconds ===========================
